# AQS and USGS Data Set Retrieval:
## Dependencies and API setup
### Goals and Target:
    1. Load A Volcano event from the USGS Data Set into a data frame or json object
    2. Retrieve and load AQS data for a timewindow around the volcano event from sensors deteremined by a bounding box of a pair of (lat,long) keys.
    3. Retrieve only records for gasses we are interested in.
    4. Set up helper functions and files to make scaling painless.

In [2]:
# Import libraries:
import datetime
from datetime import timedelta
import time
import pandas as pd
import io
import requests
import geojson
# AQS API Things
from config import apiURL, apiUser, apiPassword, aqsParams
#USGS API Things:
from config import usgsURL, usgsStart, usgsFormat, usgsMinmag, usgsConus

params = ','.join(list(aqsParams.values()))



## AQS URL Construction and helper functions:
1. Take gcs_coords and event_date from usgs earthquake api
2. Format Date/time USGS uses iso8061 time specs

In [3]:
# Define functions to create a bounding box a time window for the api call
# BBox takes an argument window which is how many square degrees our box will be:
def date_coord(df):
    date = df.Date
    coord = df.GCS
    return date, coord

def bbox(window, gcs_coords):
    bbox = {'minlat': gcs_coords[0]-window,
            'maxlat': gcs_coords[0]+window,
            'minlon': gcs_coords[1]-window,
            'maxlon': gcs_coords[1]+window}
    bbox_url = f'&minlat={bbox["minlat"]}&maxlat={bbox["maxlat"]}&minlon={bbox["minlon"]}&maxlon={bbox["maxlon"]}'
    return bbox_url
# t_window takes a datetime and a time length to return a start and end date a time length's apart
def t_window(event_date, window_length):
    window = timedelta(days=window_length)
    event_time = time.strptime(event_date, '%Y%m%d')
    event_time = datetime.datetime(*event_time[:3])
    date_window = {'bdate': (event_time - window).strftime("%Y%m%d"),
                   'edate': (event_time + window).strftime("%Y%m%d")}
    tw_url = f'&bdate={date_window["bdate"]}&edate={date_window["edate"]}'
    return tw_url

def aqs_api_call(event_date, gcs_coords):
    bbox_url = bbox(0.5, gcs_coords)
    tw_url = t_window(event_date, 7)
    aqs_url = f'{apiURL}param={params[:]}&email={apiUser}&key={apiPassword}{bbox_url}{tw_url}'
    aqs_raw = requests.get(aqs_url)
    aqs_raw = aqs_raw.json()
    return aqs_raw

def aqs_scrub(aqs_json)

## USGS URL Construction and helper functions:

In [4]:
# Define fucntions to querry USGS for earthquakes in the US with mag > 7
# Return gcs_coords(epicenter in lat,long), event_date(), magnitude, uid, depth
# In this case the parameters for the USGS data search were decided in advance and 
# are specified and hardcoded in the config file. All we need is a function
# To create the api call from the specs, and reduce the data to our target parameters:
def usgs_api_call():
    usgs_url = f"{usgsURL}{usgsFormat}{usgsStart}{usgsMinmag}{usgsConus['minlat']}{usgsConus['maxlat']}{usgsConus['minlon']}{usgsConus['maxlon']}"
    usgs_raw = requests.get(usgs_url)
    usgs_raw = usgs_raw.json()
    return usgs_raw

#Exctract the information we're interested in stuff into a dataframe
def usgs_scrub(usgs_geojson):
    target_data = []
    i=0
    for entry in usgs_geojson['features']:
        identifier = usgs_geojson['features'][i]['id']
        mag = usgs_geojson['features'][i]['properties']['mag']
        epochtime = usgs_geojson['features'][i]['properties']['time']
        converted_date= time.strftime('%Y%m%d',  time.gmtime(epochtime/1000))
        converted_time = time.strftime('%H:%M:%S',  time.gmtime(epochtime/1000))
        place = usgs_geojson['features'][i]['properties']['place']
        lon = float(usgs_geojson['features'][i]['geometry']['coordinates'][0])
        lat = float(usgs_geojson['features'][i]['geometry']['coordinates'][1])
        gcs_coords = [lat,lon]
        depth = usgs_geojson['features'][i]['geometry']['coordinates'][2]
        entry_dict = {"Identifier":identifier, 
                       "Location":place,
                       "GCS": gcs_coords, 
                       "Depth":depth,
                       "Magnitude":mag, 
                       "Date":converted_date,
                       "Time":converted_time}
        target_data.append(entry_dict)
        i+=1
    target_df = pd.DataFrame(target_data)
    return target_df

    

    

## API Calls 
1. Call USGS api first to get coordinates and date to feed into the AQS API
2. Call AQS API with coordinates and date
3. Clean AQS Data

In [5]:
# USGS Call and get date and coordinates for next call:
earthquake_df = usgs_scrub(usgs_api_call())
earthquake_df
event_time, gcs_coords = date_coord(earthquake_df)

In [ ]:
# AQS Call:
print(gcs_coords[0])
eqaq_raw = aqs_api_call(event_time[0], gcs_coords[0])
#eqaq_raw

In [7]:
earthquake_df

,Identifier,Location,GCS,Depth,Magnitude,Date,Time
0,ci38457511,2019 Ridgecrest Earthquake Sequence,"[35.7695, -117.5993333]",8.000,7.1,20190706,03:19:53
1,usp000jhr6,"Baja California, Mexico","[28.696, -113.104]",13.000,7.0,20120412,07:15:48
2,ci14607652,"12km SW of Delta, B.C., MX","[32.2861667, -115.2953333]",9.987,7.2,20100404,22:40:42
3,ci9108652,"16km SW of Ludlow, CA","[34.6033333, -116.265]",13.730,7.1,19991016,09:46:44
4,ci3031111,"Landers, California Earthquake","[34.2, -116.437]",-0.097,7.3,19920628,11:57:34
5,nc269151,"20km SSW of Rio Dell, California","[40.3353333, -124.2286667]",9.856,7.2,19920425,18:06:05
